# Daten abgreifen bei Epex Spot (4.2)

Das Problem wurde gelöst, dass bei alten Seiten leere DIV vorhanden sind. Try/Except als Lösung.

Auf der Website von Epex Sport die Preise zum Schweizer Markt abgreifen und in ein Excel exportieren. In Version 2 ergänzt um weitere Werte von der Website.

### Neue Abgreif-Funktion
Neues Problem: In den frühen Jahren gibt es noch keine Daten in den DIV für Österreich und Neues Deutschland. Die DIV bestehen schon, daher muss an der Numerierung nichts geändert werden. Jedoch muss für die Abfrage eine Sprungfunktion eingebaut werden.

Lösung: Die find_all_funktion für die TDs wird hinter die try-Anweisung gehängt.

Neues Problem: Die leeren Werte sind Strings. Damit kann nicht gerechnet werden. Mögiche Lösung: Leerschlagwerte durch NAN ersetzen.

In [1]:
#Hilfsmittel laden
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
import numpy as np

def abgreifen_neu(konkretes_datum, roher_datumswert):
    url_basis = "https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/"
    url_ende = ""
    url = url_basis + str(konkretes_datum) + url_ende
    
    epex = requests.get(url).text
    soup = BeautifulSoup(epex, "lxml")
    
    tabellendivs = soup.find_all("div", style="display: none")
    
    div_schweiz = tabellendivs[0]
    div_deutschland_neu = tabellendivs[1]
    div_deutschland_alt = tabellendivs[3]
    div_oesterreich = tabellendivs[2]

    
    nummer_preis_base = 7
    nummmer_volumen_base = 15
    nummer_preis_peak = 24
    nummer_volumen_peak = 32
    
    preise = {"datum":konkretes_datum}
    
    #Abfrage für die Schweiz
    tabellenzellen_schweiz = div_schweiz.find_all("td")
    
    #peak
    preis_peak = float(tabellenzellen_schweiz[nummer_preis_peak].text.replace(',','.'))
    preise["ch_preis_peak"]=preis_peak
    volumen_peak = float(tabellenzellen_schweiz[nummer_volumen_peak].text.replace(".","").replace(",","."))
    preise["ch_volumen_peak"]= volumen_peak
    
    #base
    preis_base = float(tabellenzellen_schweiz[nummer_preis_base].text.replace(',','.'))
    preise["ch_preis_base"] = preis_base
    volumen_base = float(tabellenzellen_schweiz[nummmer_volumen_base].text.replace(".","").replace(",","."))
    preise["ch_volumen_base"]= volumen_base    
                         

    #Abfrage für Deutschland_neu
    
    try:
        tabellenzellen_deutschland_neu = div_deutschland_neu.find_all("td")

        #peak
        de_neu_preis_peak = float(tabellenzellen_deutschland_neu[nummer_preis_peak].text.replace(',','.'))
        preise["de_neu_preis_peak"] = de_neu_preis_peak
        de_neu_volumen_peak = float(tabellenzellen_deutschland_neu[nummer_volumen_peak].text.replace(".","").replace(",","."))
        preise["de_neu_volumen_peak"]= de_neu_volumen_peak

        #base
        de_neu_preis_base = float(tabellenzellen_deutschland_neu[nummer_preis_base].text.replace(',','.'))
        preise["de_neu_preis_base"] = de_neu_preis_base
        de_neu_volumen_base = float(tabellenzellen_deutschland_neu[nummmer_volumen_base].text.replace(".","").replace(",","."))
        preise["de_neu_volumen_base"]= de_neu_volumen_base    
    except:
        preise["de_neu_preis_peak"] = ""
        preise["de_neu_volumen_peak"]= ""
        preise["de_neu_preis_base"] = ""
        preise["de_neu_volumen_base"]= ""
    
    #Anfrage für Deutschland_alt
    
    try:
        tabellenzellen_deutschland_alt = div_deutschland_alt.find_all("td")

        #peak
        de_alt_preis_peak = float(tabellenzellen_deutschland_alt[nummer_preis_peak].text.replace(',','.'))
        preise["de_alt_preis_peak"] = de_alt_preis_peak
        de_alt_volumen_peak = float(tabellenzellen_deutschland_alt[nummer_volumen_peak].text.replace(".","").replace(",","."))
        preise["de_alt_volumen_peak"]= de_alt_volumen_peak

        #base
        de_alt_preis_base = float(tabellenzellen_deutschland_alt[nummer_preis_base].text.replace(',','.'))
        preise["de_alt_preis_base"] = de_alt_preis_base
        de_alt_volumen_base = float(tabellenzellen_deutschland_alt[nummmer_volumen_base].text.replace(".","").replace(",","."))
        preise["de_alt_volumen_base"]= de_alt_volumen_base    
    except:
        preise["de_alt_preis_peak"] = ""
        preise["de_alt_volumen_peak"]= ""
        preise["de_alt_preis_base"] = ""
        preise["de_alt_volumen_base"]= ""

    #Anfrage für Oesterreich
    
    try:
        tabellenzellen_oesterreich = div_oesterreich.find_all("td")

        #peak
        at_preis_peak = float(tabellenzellen_oesterreich[nummer_preis_peak].text.replace(',','.'))
        preise["at_preis_peak"] = at_preis_peak
        at_volumen_peak = float(tabellenzellen_oesterreich[nummer_volumen_peak].text.replace(".","").replace(",","."))
        preise["at_volumen_peak"]= at_volumen_peak

        #base
        at_preis_base = float(tabellenzellen_oesterreich[nummer_preis_base].text.replace(',','.'))
        preise["at_preis_base"] = at_preis_base
        at_volumen_base = float(tabellenzellen_oesterreich[nummmer_volumen_base].text.replace(".","").replace(",","."))
        preise["at_volumen_base"]= at_volumen_base    
    except:
        preise["at_preis_peak"] = ""
        preise["at_volumen_peak"]= ""
        preise["at_preis_base"] = ""
        preise["at_volumen_base"]= ""        
        
    #Druckdatum erstellen
    try:
        druckdatum = roher_datumswert.strftime("%d.%m.%Y")
        preise["00 druckdatum"] = druckdatum
    except:
        None
        
    #print(preise)
    print(url)
    return preise
        
abgreifen_neu("2010-08-08", "2010-08-08")



https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2010-08-08


{'datum': '2010-08-08',
 'ch_preis_peak': 28.83,
 'ch_volumen_peak': 11718.5,
 'ch_preis_base': 25.07,
 'ch_volumen_base': 26519.1,
 'de_neu_preis_peak': '',
 'de_neu_volumen_peak': '',
 'de_neu_preis_base': '',
 'de_neu_volumen_base': '',
 'de_alt_preis_peak': 31.4,
 'de_alt_volumen_peak': 248269.6,
 'de_alt_preis_base': 25.97,
 'de_alt_volumen_base': 471108.8,
 'at_preis_peak': '',
 'at_volumen_peak': '',
 'at_preis_base': '',
 'at_volumen_base': ''}

## Schleife
Schleife erstellen, die für einen bestimmten Datenrange alle Datenpaare abfragt. Die Schleife macht aus jedem Datenpaar ein Dictionary und schreibt dieses in die Liste "preisliste".

In [2]:
#Neue Schleife, basierend auf abgreifen_neu
preisliste_neu = []

start = "2018-01-09"
ende = "2018-11-02"

#erstelle Datenrange
import datetime
start_datum = datetime.datetime.strptime(start, "%Y-%m-%d")
end_datum = datetime.datetime.strptime(ende, "%Y-%m-%d")
date_generated = [start_datum + datetime.timedelta(days=x) for x in range(0, (end_datum-start_datum).days)]

#schleife für abfrage
for date in date_generated:
    #print(date.strftime("%Y-%m-%d"))
    abgreifdatum = date.strftime("%Y-%m-%d")
    datensatz = abgreifen_neu(abgreifdatum, date)
    preisliste_neu.append(datensatz)

print(preisliste_neu)

https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-09
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-10
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-11
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-12
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-13
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-14
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-15
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-16
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-17
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-18
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-19
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-01-20
https://www.epexspot.com/de/marktdaten/d

https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-22
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-23
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-24
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-25
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-26
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-27
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-28
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-29
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-04-30
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-05-01
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-05-02
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-05-03
https://www.epexspot.com/de/marktdaten/d

https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-03
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-04
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-05
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-06
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-07
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-08
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-09
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-10
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-11
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-12
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-13
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2018-08-14
https://www.epexspot.com/de/marktdaten/d

## Analyse in Panda

In [3]:
#Pandas laden
import pandas as PD
%matplotlib inline

datensatz = pd.DataFrame(preisliste_neu)
datensatz.dtypes
#datensatz.plot()

00 druckdatum           object
at_preis_base           object
at_preis_peak           object
at_volumen_base         object
at_volumen_peak         object
ch_preis_base          float64
ch_preis_peak          float64
ch_volumen_base        float64
ch_volumen_peak        float64
datum                   object
de_alt_preis_base       object
de_alt_preis_peak       object
de_alt_volumen_base     object
de_alt_volumen_peak     object
de_neu_preis_base       object
de_neu_preis_peak       object
de_neu_volumen_base     object
de_neu_volumen_peak     object
dtype: object

## Leerwerte durch NAN ersetzen
Der folgende Code ersetzt die oben generierten Leerschläge durch saubere NAN werte, damit mit den Daten gerechnet werden kann.

In [4]:
import numpy as np
#datensatz['de_neu_volumen_peak'] = datensatz['de_neu_volumen_peak'].replace('', np.nan)
datensatz = datensatz.replace('', np.nan)
datensatz.dtypes

00 druckdatum           object
at_preis_base          float64
at_preis_peak          float64
at_volumen_base        float64
at_volumen_peak        float64
ch_preis_base          float64
ch_preis_peak          float64
ch_volumen_base        float64
ch_volumen_peak        float64
datum                   object
de_alt_preis_base      float64
de_alt_preis_peak      float64
de_alt_volumen_base    float64
de_alt_volumen_peak    float64
de_neu_preis_base      float64
de_neu_preis_peak      float64
de_neu_volumen_base    float64
de_neu_volumen_peak    float64
dtype: object

## Alternative: Daten aus CSV laden

In [ ]:
#Pandas laden
import pandas as pd
%matplotlib inline


dateiname = "20181028_2007-2018_zusammengesetzt.csv"
datensatz = pd.read_csv(dateiname)

datensatz

## Mit Pandas neue Datenreihen berechnen

Die Datenreihen Deutschland alt und Deutschland neu sollen in eine gemeinsame Datenreihe vereint werden. Bei den Preisen geschieht das durch addition, was möglich ist, weil jeweils eine der Datenreihen immer leer ist. Beim Volumen wird annalog vorgegangen. Die neue Deutschland-Reihe könnte dort enen Klick aufweisen, da ab Oktober 2018 Österreich nicht mehr darin enthalten ist. Da Österreich vermutlich ein Preisnehmer ist, dürfte sich das Ausscheren kaum auf den preis in Deutschland auswirken. 

In [6]:
# 2. Durchgängige Zeitreihen für Deutschland erstellen: Altes Deutschland/At & Neues Deutschland/Lux 
# Die neue Datenreihe besteht als derjenigen zusammen


#Problem: NAN können nicht mit Float64 addiert werden. Daher müssen die NAN vor der Addition durch Null ersetzt werden.
#Funktion .fillna(0)


#Neue Reihe für Deutschland
try:
    datensatz["de_kombi_preis_peak"] = datensatz["de_alt_preis_peak"].fillna(0) + datensatz["de_neu_preis_peak"].fillna(0)
    datensatz["de_kombi_preis_base"] = datensatz["de_alt_preis_base"].fillna(0) + datensatz["de_neu_preis_base"].fillna(0)
    datensatz["de_kombi_spread"] = datensatz["de_kombi_preis_peak"].fillna(0) - datensatz["de_kombi_preis_base"].fillna(0)
except:
    print("Problem mit Deutschland")

#Neue Reihe für Österreich
try:
    datensatz["at_kombi_preis_peak"] = datensatz["de_alt_preis_peak"].fillna(0) + datensatz["at_preis_peak"].fillna(0)
    datensatz["at_kombi_preis_base"] = datensatz["de_alt_preis_base"].fillna(0) + datensatz["at_preis_base"].fillna(0)
    datensatz["at_kombi_spread"] = datensatz["at_kombi_preis_peak"].fillna(0) - datensatz["at_kombi_preis_base"].fillna(0)
except:
    print("Problem mit Österreich")
datensatz.dtypes
  

00 druckdatum           object
at_preis_base          float64
at_preis_peak          float64
at_volumen_base        float64
at_volumen_peak        float64
ch_preis_base          float64
ch_preis_peak          float64
ch_volumen_base        float64
ch_volumen_peak        float64
datum                   object
de_alt_preis_base      float64
de_alt_preis_peak      float64
de_alt_volumen_base    float64
de_alt_volumen_peak    float64
de_neu_preis_base      float64
de_neu_preis_peak      float64
de_neu_volumen_base    float64
de_neu_volumen_peak    float64
de_kombi_preis_peak    float64
de_kombi_preis_base    float64
de_kombi_spread        float64
at_kombi_preis_peak    float64
at_kombi_preis_base    float64
at_kombi_spread        float64
dtype: object

## Spreads brechnen

Zu jedem Land sollen die Spreads zwischen Spitzenstrom und Bandstrom berechnet und als eigene Reihe "*_spread" abgespeichert werden. Bei den neu berechneten Reihen zu DE und AT ist das bereits im Schritt oben schon geschehen.

In [7]:
# 2. Spreads einbauen
datensatz["ch_spread"] = datensatz["ch_preis_peak"]-datensatz["ch_preis_base"]
datensatz["at_spread"] = datensatz["at_preis_peak"]-datensatz["at_preis_base"]
datensatz["de_alt_spread"] = datensatz["de_alt_preis_peak"]-datensatz["de_alt_preis_base"]
datensatz["de_neu_spread"] = datensatz["de_neu_preis_peak"]-datensatz["de_neu_preis_base"]

# 5. Für jeden Tag den Aufpreis zwischen Schweiz und Deutschland berechnen


In [8]:
# 3. Preisdifferenzen zwischen den Ländern einbauen: Schweiz, Österreich - für Bandstrom
# Berechnung als gleitender Mittelwert, um kurzfristige Ausreisser zu glätten

# Definition des gleitenden Mittelwerts
gleiter = 31

datensatz["ch_aufpreis_base_rolling"] = (datensatz["ch_preis_base"] - datensatz["de_kombi_preis_base"]).rolling(gleiter).mean()
datensatz["at_aufpreis_base_rolling"] = (datensatz["at_kombi_preis_base"] - datensatz["de_kombi_preis_base"]).rolling(gleiter).mean()
datensatz["ch_aufpreis_peak_rolling"] = (datensatz["ch_preis_peak"] - datensatz["de_kombi_preis_peak"]).rolling(gleiter).mean()
datensatz["at_aufpreis_peak_rolling"] = (datensatz["at_kombi_preis_peak"] - datensatz["de_kombi_preis_peak"]).rolling(gleiter).mean()

Eigene Funktion: Diese neu geschaffenen Spalten mit den Aufpreisen in ein neues DataFrame exportieren

In [ ]:
dateiname = "20181028_export_aufpreis_2017-2019"
spaltenliste = ["00 druckdatum","ch_aufpreis_base_rolling","at_aufpreis_base_rolling","ch_aufpreis_peak_rolling","at_aufpreis_peak_rolling"]
datensatz[spaltenliste].to_excel(dateiname+".xls", index=False)
datensatz[spaltenliste].to_csv(dateiname+".csv", index=False)

## Gleitende Durchschnitte
Zu jedem Land gleitende Durchschnitte berechnen und als eigene Datei exportieren.

In [11]:
# setzt voraus, dass für Österreich und Deutschland durchgehende Reihen berechnet wurden
# Definition des gleitenden Mittelwerts
gleiter = 30

#datensatz["ch_preis_base_rolling_y"] = datensatz["ch_preis_base"].rolling(365).mean()
#datensatz["ch_preis_peak_rolling_y"] = datensatz["ch_preis_peak"].rolling(365).mean()

datensatz["ch_preis_base_rolling"] = datensatz["ch_preis_base"].rolling(gleiter).mean()
datensatz["ch_preis_peak_rolling"] = datensatz["ch_preis_peak"].rolling(gleiter).mean()
datensatz["de_kombi_preis_base_rolling"] = datensatz["de_kombi_preis_base"].rolling(gleiter).mean()
datensatz["de_kombi_preis_peak_rolling"] = datensatz["de_kombi_preis_peak"].rolling(gleiter).mean()
datensatz["at_kombi_preis_base_rolling"] = datensatz["at_kombi_preis_base"].rolling(gleiter).mean()
datensatz["at_kombi_preis_peak_rolling"] = datensatz["at_kombi_preis_peak"].rolling(gleiter).mean()

In [12]:
#Diese Daten einzeln exportieren
dateiname = "2018 rollende Preise alleine"

spaltenliste = [
    "00 druckdatum",
    "ch_preis_base_rolling",
    #"ch_preis_base_rolling_y",
    "ch_preis_peak_rolling",
    #"ch_preis_peak_rolling_y",
    "de_kombi_preis_base_rolling",
    "de_kombi_preis_peak_rolling",
    "at_kombi_preis_base_rolling",
    "at_kombi_preis_peak_rolling",
]
datensatz[spaltenliste].to_excel(dateiname+".xls", index=False)
datensatz[spaltenliste].to_csv(dateiname+".csv", index=False)

## Export alles in CSV

In [13]:
dateiname = "2018 alle Daten"
datensatz.to_csv(dateiname+".csv", index=False)
datensatz.to_excel(dateiname+".xls", index=False)